In [27]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [28]:
def get_bins(df, column, n_bins):
    min_v = df[column].min()
    max_v = df[column].max()
    array = []
    
    for i in range(n_bins+1):
        step = max_v/n_bins
        array.append(int(round(step*i, 0)))
    
    return array

In [29]:
## Get the names and open data
data_domicilio_path = ('./data/CENSO/Domicilio02_PB.csv') 

with open(data_domicilio_path, "r", encoding="latin-1") as myfile:
    for i in range(1):
        line = next(myfile).strip()
        names = line.split(';')


data = pd.read_csv(data_domicilio_path,                  
                sep=';', 
                header=None, 
                names=names,
                skiprows=1,
                encoding="latin-1",
                dtype={'Cod_setor': str})

In [42]:
data['Cod_setor']

0       250010605000001
1       250010605000002
2       250010605000003
3       250010605000004
4       250010605000005
             ...       
5543    251720905000004
5544    251720905000005
5545    251740705000001
5546    251740705000002
5547    251740705000003
Name: Cod_setor, Length: 5548, dtype: object

In [36]:
data.query('Cod_setor == 251720905000004')

,Cod_setor,Situacao_setor,V001,V002,V003,V004,V005,V006,V007,V008,...,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132


In [30]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [31]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [32]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
moradores = data[['Cod_setor', 'V001']]
moradores = moradores.rename(columns={'V001': 'MORADORES'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [33]:
# Set the same inde
moradores.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [34]:
moradores.to_csv('./data/POPULACAO/populacao_parse.csv')

In [20]:
setores = setores.join(moradores)

In [21]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [22]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [11]:
def get_area(row):
    return row['geometry'].area/10000

def get_density(row):
    density = row['MORADORES']/row['area']
    return density

setores_recorte['area'] = setores_recorte.apply(get_area, axis=1)
setores_recorte['densidade'] = setores_recorte.apply(get_density, axis=1)
bairros['area'] = bairros.apply(get_area, axis=1)
bairros['densidade'] = bairros.apply(get_density, axis=1)

<ipython-input-11-a00a87234b51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['area'] = setores_recorte.apply(get_area, axis=1)
<ipython-input-11-a00a87234b51>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['densidade'] = setores_recorte.apply(get_density, axis=1)


In [12]:
setores_recorte.to_file('./data/POPULACAO/setores_recorte.shp')
bairros.to_file('./data/POPULACAO/bairros.shp')

,geometry,NM_BAIRRO
CD_GEOCODI,,
250750705000153,"POLYGON ((-3880337.824 -793550.572, -3880322.8...",Estados
250750705000154,"POLYGON ((-3880206.504 -793549.049, -3880131.9...",Estados
250750705000155,"POLYGON ((-3879433.625 -794026.389, -3879502.9...",Estados
250750705000156,"POLYGON ((-3879858.613 -794140.257, -3879860.2...",Estados
250750705000157,"POLYGON ((-3880720.755 -793909.100, -3880719.7...",Estados
...,...,...
250750705000959,"POLYGON ((-3880279.855 -792938.669, -3880133.0...",Ipês
250750705000960,"POLYGON ((-3880426.450 -792975.127, -3880279.8...",Ipês
250750705000961,"POLYGON ((-3877284.915 -804320.231, -3877232.0...",Paratibe
